In [10]:
import os
import pandas as pd
import json
from utils.statistics import *
from utils.preprocess import *
import shutil

In [11]:
source_root = "/home/techt/Desktop/AI_for_Science/archiv/gold_icml"
dest_root = "/home/techt/Desktop/a4s/output/preprocess/no_expert"  # specify

In [12]:
def org_normalize_trait(trait):
    trait = '-'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

In [13]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]
all_org_nm_traits = [org_normalize_trait(at) for at in all_traits]

### Add Xena data

In [14]:
s_ls = []
for t in os.listdir(dest_root):
    s_path = os.path.join(source_root, t)
    if not os.path.isdir(s_path): continue
    if not os.path.isdir(os.path.join(dest_root, t)):
        s_ls.append(f"Strange! {t} not in {dest_root}")
        continue
    for f in os.listdir(s_path):
        if f.endswith('Xena.csv'):
            source_fpath = os.path.join(s_path, f)
            if os.path.isfile(os.path.join(dest_root, t, 'Xena.csv')):
                print(f"{os.path.join(dest_root, t, 'Xena.csv')} exist corresponding to {source_fpath}")
            else:
                print(f"Copied because {os.path.join(dest_root, t, 'Xena.csv')} not exist corresponding to {source_fpath}")
                shutil.copyfile(os.path.join(source_root, t, 'Xena.csv'), os.path.join(dest_root, t, 'Xena.csv'))

Copied because /home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/Xena.csv not exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Thyroid_Cancer/Xena.csv
Copied because /home/techt/Desktop/a4s/output/preprocess/no_expert/Breast_Cancer/Xena.csv not exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Breast_Cancer/Xena.csv
Copied because /home/techt/Desktop/a4s/output/preprocess/no_expert/Stomach_Cancer/Xena.csv not exist corresponding to /home/techt/Desktop/AI_for_Science/archiv/gold_icml/Stomach_Cancer/Xena.csv


OSError: [Errno 28] No space left on device: '/home/techt/Desktop/AI_for_Science/archiv/gold_icml/Stomach_Cancer/Xena.csv' -> '/home/techt/Desktop/a4s/output/preprocess/no_expert/Stomach_Cancer/Xena.csv'

In [ ]:
s_ls

### Remove data with missing gene, remove dummy column

In [15]:

for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_org_nm_traits:
            n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
            os.rename(os.path.join(dest_root, trait), os.path.join(dest_root, n_trait))
            print(f"'{trait}' trait has been newly normalized to {n_trait} in the dir name")
            trait = n_trait
        else:
            print(f"!Strange trait'{trait}' ")
            continue
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    js_path = os.path.join(trait_dir, "cohort_info.json")
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            cohort = fn[:-4]
            df = pd.read_csv(fp)
            if 'Unnamed: 0' in df.columns:
                print(f"REDUNDANT COLUMN in '{fp}'")
                df = df.drop(columns = ['Unnamed: 0'], errors="ignore")
                df.to_csv(fp, index=False)
            header = df.columns[0]
            if header not in all_nm_traits:
                if header in all_traits:
                    nm_trait = all_nm_traits[all_traits.index(header)]
                    print(f"The header {header} should be replaced by {nm_trait} in '{fn}'")
                    df.columns = [nm_trait] + df.columns[1:].tolist()
                    df.to_csv(fp, index=False)
                    print('This cohort updated')
                else:
                    print(f"Strange header {header}")

            if len(df.columns) <= 4:
                print(f"NO GENE DATA in '{fp}'. Removed")
                os.remove(fp)
                if os.path.isfile(js_path):
                    with open(js_path, 'r') as file:
                        info = json.load(file)
                    try:
                        save_cohort_info(cohort, js_path, False, info[cohort]['is_trait_available'], info[cohort]['is_biased'])
                        print("!Lack of gene updated in info.json")
                    except KeyError as e:
                        print(e)
                        continue

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
REDUNDANT COLUMN in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Tourette_Syndrome/GSE146509.csv'
The header Thyroid Cancer should be replaced by Thyroid_Cancer in 'Xena.csv'
This cohort updated
REDUNDANT COLUMN in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/GSE104005.csv'
REDUNDANT COLUMN in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/GSE104006.csv'
NO GENE DATA in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/GSE104006.csv'. Removed
!Lack of gene updated in info.json
REDUNDANT COLUMN in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/GSE151181.csv'
NO GENE DATA in '/home/techt/Desktop/a4s/output/preprocess/no_expert/Thyroid_Cancer/GSE151181.csv'. Removed
!Lack of gene updated in info.json
REDUNDANT COLUMN in '/home/techt/Desktop/a4s/output/preproce

In [16]:
for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_org_nm_traits:
            n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
            os.rename(os.path.join(dest_root, trait), os.path.join(dest_root, n_trait))
            print(f"'{trait}' trait has been newly normalized to {n_trait} in the dir name")
            trait = n_trait
        else:
            print(f"!Strange trait'{trait}' ")
            continue
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    js_path = os.path.join(trait_dir, "cohort_info.json")
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            cohort = fn[:-4]
            df = pd.read_csv(fp)
            if 'Unnamed: 0' in df.columns:
                print(f"REDUNDANT COLUMN in '{fp}'")
                df = df.drop(columns = ['Unnamed: 0'], errors="ignore")
                df.to_csv(fp, index=False)
            header = df.columns[0]
            if header not in all_nm_traits:
                if header in all_traits:
                    nm_trait = all_nm_traits[all_traits.index(header)]
                    print(f"The header {header} should be replaced by {nm_trait} in '{fn}'")
                    df.columns = [nm_trait] + df.columns[1:].tolist()
                    df.to_csv(fp, index=False)
                    print('This cohort updated')
                else:
                    print(f"Strange header {header}")

            if len(df.columns) <= 4:
                print(f"NO GENE DATA in '{fp}'. Removed")
                os.remove(fp)
                if os.path.isfile(js_path):
                    with open(js_path, 'r') as file:
                        info = json.load(file)
                    try:
                        save_cohort_info(cohort, js_path, False, info[cohort]['is_trait_available'], info[cohort]['is_biased'])
                        print("!Lack of gene updated in info.json")
                    except KeyError as e:
                        print(e)
                        continue

!Strange trait'last_cohort_info.json' 
Strange header Alcohol_Dependence
Strange header Group


### Update json

In [17]:
for trait in os.listdir(dest_root):
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    js_path = os.path.join(trait_dir, "cohort_info.json")
    if not os.path.isfile(js_path):
        print(f"Strange! {js_path} not exist.")
        continue
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("Xena")):
            cohort = "Xena"
            df = pd.read_csv(fp)
            header = df.columns[0]
            if header not in all_nm_traits:
                print(f'strange header {header}')
                continue
            trait_biased, _ = judge_and_remove_biased_features(df, header)
            save_cohort_info(cohort, js_path, True, True, trait_biased, df)
            print(f"Trait {trait} info json updated with cohort {cohort}, biased:{trait_biased}")


For the feature 'Thyroid_Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Trait Thyroid_Cancer info json updated with cohort Xena, biased:False
Strange! /home/techt/Desktop/a4s/output/preprocess/no_expert/DNA_Repair_Capacity/cohort_info.json not exist.
Strange! /home/techt/Desktop/a4s/output/preprocess/no_expert/Rheumatoid_Arthritis/cohort_info.json not exist.
For the feature 'Breast_Cancer', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dat

In [18]:
rm -rf regress

nohup python regress.py --version gs2 >> gs2.log 2>&1 &
nohup python regress.py --version gs1 >> gs1.log 2>&1 &
nohup python regress.py --version ours >> ours.log 2>&1 &
nohup python regress.py --version no_expert >> ne.log 2>&1 &
nohup python regress.py --version round0 >> r0.log 2>&1 &

SyntaxError: invalid syntax (793577071.py, line 1)

In [19]:
import pandas as pd
import os
import json
import ast
import argparse
from utils.statistics import *

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('--version', type=str, help='Specify the version.')

args = parser.parse_args()

pairs = pd.read_csv("refined_pairs.csv")


usage: ipykernel_launcher.py [-h] [--version VERSION]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/techt/.local/share/jupyter/runtime/kernel-054d2d0e-1461-4f85-88a1-f2940b909d62.json


SystemExit: 2

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
avail = []
for trait in all_nm_traits:
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    for c in os.listdir(trait_dir):
        if c.endswith(".csv") and (c.startswith("GSE") or c.startswith("Xena")):
            avail.append(trait)
            break

d = pd.read_csv("trait_related_genes.csv")["Trait"].tolist()
missing = [a for a in avail if a not in d]


In [ ]:
traits = [all_traits[all_nm_traits.index(t)] for t in missing]

In [ ]:
traits

In [ ]:
for i, t in enumerate(traits):
    print(i+1, t)

In [ ]:
traits

In [ ]:
missing

In [ ]:
len(traits), len(missing)

In [ ]:
from ast import literal_eval
df = pd.read_csv("trait_related_genes.csv")
df['Related_Genes'] = df['Related_Genes'].apply(literal_eval)
tr_genes = df.to_dict(orient='records')

In [ ]:
data_dir = './downloaded_ncbi_genes'
file_names = os.listdir(data_dir)

def extract_number(file_name):
    start = file_name.find('(') + 1  # find the index of the opening parenthesis
    end = file_name.find(')')        # find the index of the closing parenthesis
    return int(file_name[start:end]) if start > 0 and end > 0 else 0

# Sort the file names using the extracted number as the key
sorted_file_names = sorted(file_names, key=extract_number)

for trait, file in zip(missing, sorted_file_names):
    file_path = os.path.join(data_dir, file)
    try:
        df = pd.read_csv(file_path, sep='\t', header=0)  # header=0 indicates the first line contains column headers

        gene_symbols = df['Symbol'].tolist()
        gene_symbols = normalize_gene_symbols(gene_symbols)

        unique_symbols = []
        for item in gene_symbols:
            if item not in unique_symbols:
                unique_symbols.append(item)

        tr_genes.append({'Trait': trait, 'Related_Genes': unique_symbols})
        print(trait, len(unique_symbols))
    except:
        print(f"Exception in file{file}")

gene_df = pd.DataFrame(tr_genes)

In [ ]:
gene_df.to_csv("Trait_related_genes_new.csv", index=False)

In [ ]:
avail = []
for trait in all_nm_traits:
    trait_dir = os.path.join(dest_root, trait)
    if not os.path.isdir(trait_dir):
        continue
    for c in os.listdir(trait_dir):
        if c.endswith(".csv") and (c.startswith("GSE") or c.startswith("Xena")):
            avail.append(trait)
            break

In [ ]:
len(avail)

In [ ]:
al = []
for t in avail:
    al.append({"Trait": t})

In [ ]:
al

In [ ]:
len(al)

In [ ]:
latest_task = pd.DataFrame(al)

In [ ]:
latest_task.to_csv("latest_task.csv", index = False)

In [ ]:
batch_size = 9
gene_symbols = list(range(10))
for i in range(0, len(gene_symbols), batch_size):
    batch = gene_symbols[i:i + batch_size]
    print(batch)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[9]
